# Chapter 8: Calculation of Molecular Properties

## 8.1. Calculation of Potential Energy of Neutral Molecules

Computational methods allows the calculation of potential energy of molecules. From energy calculation, many molecular properties can be derived such as electron density, dipole moment, and energy of molecular orbitals

In the following section, we will calculate the potential energies of benzene and substituted benzene compounds:

### 8.1.1. Calculation of Potential Energy

In [ ]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from utils import View3DModel
import psi4
import py3Dmol
from tqdm import tqdm

In [ ]:
# Create the core molecule
core_mol = Chem.MolFromSmiles('c1ccccc1*') # Add * next to the atom you want to attach the substituents
core_mol

In [ ]:
# Define substituents
substituents = ['[H]', 'F', 'Cl', 'C', 'O', 'N', 'OC', 'C(=O)', 'C(=O)O', 'C(=O)OC', 'C(=O)N(C)C', 'C#N']

In [ ]:
# Generate substituted benzenes
substituted_benzenes = []

for substituent in substituents:
    # Create a copy of the core molecule
    core_mol_copy = Chem.Mol(core_mol)

    # Replace a hydrogen atom with the substituent
    subst_mol = Chem.MolFromSmiles(substituent)
    subst_mol_smiles = Chem.MolToSmiles(Chem.rdmolops.ReplaceSubstructs(core_mol_copy, Chem.MolFromSmarts('[#0]'), subst_mol)[0])
    substituted_benzenes.append(Chem.MolFromSmiles(subst_mol_smiles))
    
Draw.MolsToGridImage(substituted_benzenes)

In [ ]:
# View 3D model of a molecule
mol = substituted_benzenes[2]
View3DModel(mol)

In [ ]:
# Set the number of threads and set memory limit
psi4.set_num_threads(8)
psi4.set_memory(8*1024*1024*1024) # 8 GB

In [ ]:
# Set calculation options
psi4.set_options({
    'BASIS': '6-31G*',
    'SCF_TYPE': 'DF',
    'REFERENCE': 'RHF'  # RHF for closed-shell molecules; 'UHF' or 'ROHF' for open-shell
})

In [ ]:
substituted_benzenes_geometries = []
substituted_benzenes_energies = []
substituted_benzenes_wfns = []

# Optimize the geometries and calculate the energies for all molecules
progress_bar = tqdm(substituted_benzenes)
for mol in progress_bar:
    smiles = Chem.MolToSmiles(mol)
    progress_bar.set_description(f'SMILES: {smiles}. Optimizing geometry...')
    
    # Prepare the molecule
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol, maxIters=200)
    
    # Write the geometry to XYZ string
    xyz_string = Chem.MolToXYZBlock(mol)

    # Get the psi4 geometry
    geometry = psi4.geometry(xyz_string)
    
    # Run geometry optimization
    max_iters = 500
    energy, wfn = psi4.optimize('b3lyp', molecule=geometry, optking__geom_maxiter=max_iters, return_wfn =True)
    substituted_benzenes_geometries.append(geometry)
    substituted_benzenes_energies.append(energy * psi4.constants.hartree2kcalmol)
    substituted_benzenes_wfns.append(wfn)

In [ ]:
plt.bar(range(1, len(substituted_benzenes_energies) + 1), 
        substituted_benzenes_energies)
plt.xlabel('Compound')
plt.ylabel('Potential energy (kcal/mol)')

### 8.1.2. Effect of Solvents

In the following section, we will calculate the potential energies of toluene in a solvent and derive the solvation energy using polarizable continuum model (PCM). For more details about PCM, see [documentation](https://pcmsolver.readthedocs.io/en/latest/users/input.html#medium-section-keywords).

In [ ]:
# Get the optimized geometry of toluene
toluene_idx = 3
toluene_geometry = substituted_benzenes_geometries[toluene_idx]

In [ ]:
# Set up PCM solver
pcm_string = """
    Units = Angstrom
    Medium {
        SolverType = IEFPCM
        Solvent = Water
    }
    Cavity {
       RadiiSet = UFF
       Type = GePol
       Scaling = False
       Area = 0.3
       Mode = Implicit
    }
"""

psi4.pcm_helper(pcm_string)
psi4.set_options({'pcm': True, 'pcm_scf_type': 'total'})

In [ ]:
# Calculate potential energy
toluene_energy_solvent = psi4.energy('b3lyp', molecule=toluene_geometry)
toluene_energy_solvent *= psi4.constants.hartree2kcalmol

print(f'Energy of toluene in solvent (PCM):  {toluene_energy_solvent:.2f} kcal/mol')
print(f'Solvation energy: {(toluene_energy_solvent - substituted_benzenes_energies[toluene_idx]):.2f} kcal/mol')

### 8.1.3. Visualization of Dipole Moment

From the wavefunction, we can get the dipole movement of the molecule

In [ ]:
# For example, get the dipole moment of chlorobenzene (index = 2)
mol_idx = 2
mol = substituted_benzenes[mol_idx]
dipole_moment = substituted_benzenes_wfns[mol_idx].variable("CURRENT DIPOLE")

print(f"Dipole moment (Debye): {dipole_moment}")

dipole_magnitude = np.linalg.norm(dipole_moment)
print(f"Dipole moment magnitude (Debye): {dipole_magnitude}")

# Visualize the molecule
view = py3Dmol.view(width=800, height=400)
view.addModel(Chem.MolToMolBlock(mol), "molecule", {'keepH': True})
view.setBackgroundColor('white')
view.setStyle({'stick': {'scale': 0.3}, 'sphere': {'scale': 0.3}})

# Scale the dipole for visualization purposes
scale_factor = 5
dipole_end_point = [d * scale_factor for d in dipole_moment] 

# Visualize dipole moment
view.addArrow({
    'start': {'x': 0, 'y': 0, 'z': 0},  # Starting at the origin
    'end': {'x': dipole_end_point[0], 'y': dipole_end_point[1], 'z': dipole_end_point[2]},
    'radius': 0.1,
    'fromCap': 1,
    'toCap': 1,
    'color': 'blue'
})

view.zoomTo()
view.show()

You can see that the dipole moment point from positive charge to the negative charge, which is opposite of what we learned in organic chemistry. This is because the direction of a dipole moment vector in molecular simulations is conventionally taken from the positive to the negative center of charge.

### 8.1.4. Visualization of Electron Density

After the energy calculation, you'll have the electron density data available. Psi4 can export this data in a format that can be visualized, such as a cube file.

In [ ]:
# Set options for cube file generation
psi4.set_options({'CUBEPROP_TASKS': ['DENSITY'],
                  'CUBIC_GRID_SPACING': [0.1, 0.1, 0.1],
                  'CUBEPROP_FILEPATH': './'})

# Generate the cube file
psi4.cubeprop(substituted_benzenes_wfns[mol_idx])

The cube files generated by Psi4 with the names "Da", "Db", "Dt", and "Ds" represent different types of electron densities. Here's what each one typically stands for:

- **Da (Alpha Electron Density):** This file represents the density of alpha electrons (spin-up electrons) in your molecule.

- **Db (Beta Electron Density):** This file contains the density of beta electrons (spin-down electrons). In molecules without unpaired electrons (closed-shell systems), this will be the same as the alpha electron density.

- **Dt (Total Electron Density):** This file represents the total electron density, which is the sum of the alpha and beta electron densities. For most general purposes, especially in closed-shell systems like a water molecule, this is the file you would use to visualize the overall electron density.

- **Ds (Spin Density):** This file shows the spin density, which is the difference between the alpha and beta electron densities. It's useful for visualizing unpaired electrons in open-shell systems. For a molecule like water, which is a closed-shell molecule, the spin density would typically be near zero.

For visualizing the total electron density of a molecule, you would most likely be interested in the Dt (Total Electron Density) file.

These cube files can be visualized with software such as VMD or PyMol. You can also used cube file viewer extension to view the cube files inside the working directory.

jupyterlab-cube is a JupyterLab renderer for cube files. To install jupyterlab-cube, run the following commands:

In [ ]:
!pip install jupyterlab-cube

### 8.1.5. Visualization of Electrostatic Potential Surface

After the energy calculation, cube file for electrostatic potential of the molecule can also be generated. Note that it will generate a new cube file named Dt.cube, which may may override the cube file for total electron density.  

In [ ]:
# Set options for cube file generation
psi4.set_options({'CUBEPROP_TASKS': ['esp'],
                  'CUBIC_GRID_SPACING': [0.1, 0.1, 0.1],
                  'CUBEPROP_FILEPATH': './'})

# Generate the cube file
psi4.cubeprop(substituted_benzenes_wfns[mol_idx])